In [1]:
import pandas as pd
import numpy as np

import jieba
import jieba.posseg as pseg
import jieba.analyse

import sys
from optparse import OptionParser
import pprint
from prettytable import PrettyTable

In [2]:
df = pd.read_csv(open('../data/df_small.csv'))
df = df[df['text'].str.contains("https|Britain")== False]
print('有{}筆對話'.format(len(df)))

有91340筆對話


## groupby

In [3]:
grouped_data = df.groupby('from')['text'].apply(lambda x: '/'.join(x)).reset_index()
l = len(grouped_data)
print('有{}個使用者'.format(l))

有1019個使用者


## 先用前1000筆資料做測試

In [4]:
test = grouped_data.head(1000)
test_len = len(test)
test.head()

,from,text
0,1000970273334199,找不到阿/安安/你有打LOL嗎/好吧，可以帶我嗎/安安/凸/Sor/嗯/約哪/有啊/你先約的...
1,1001070263323689,!?/唐寶寶/你那麼愛慕他ㄇ/嗨/先問問您期望本人的性別是?/了解/我是男的/抱歉沒有/請問...
2,1001967239901904,早/在床上發懶/爆/斷/魂/你在暗諷綠藻嘛/壞/話說我好餓ㄛ/窩不要這個qq/ㄋ早餐吃什麼/...
3,1003172849781848,甲/你要做我的0？/人生勝利組是不會來玩這個的/你一定不是c銘/也不是業王/葉你妹/被盜/其...
4,1003929993039131,吃噓/肥宅早/我們不是場外人/場外ㄈㄓ/我沒用了/安安/是喔/場外ㄈㄓ嗎/豪/斷/欸/在打手...


In [56]:
for i in range(len(grouped_data)):
    grouped_data.loc[i,'text'] = grouped_data.iloc[i][1].split('/')
# grouped_data.head()

In [57]:
print('id為 {} 的使用者傳送的訊息{}'.format(grouped_data.iloc[0][0],grouped_data.iloc[0][1]))

id為 1000970273334199 的使用者傳送的訊息['找不到阿', '安安', '你有打LOL嗎', '好吧，可以帶我嗎', '安安', '凸', 'Sor', '嗯', '約哪', '有啊', '你先約的啊', '喔', '輸入0088', '我懶的打了', '皮米屌', '這你嗎？', '魏癢癢', '哪裡人?', '住哪裡啦？', '超癢', '哪裡人?', '好遠', '其實還好', '新竹', '龍潭?', 'ㄏㄅㄨㄐㄅㄌㄗㄋ', '三小', '幹又ㄋ', '掰掰', '凹?', '凸sor']


In [58]:
stopwordset = set()
with open('jieba_txt/stop_words.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
# stopwordset

In [59]:
chars = set(' abcdefghijklmnopqrstuvwxyz0123456789()?!$,:.。，-_&=/􀆿\r\n')
for c in chars:
    stopwordset.add(c)

In [9]:
# stopwordset

## gensim model

In [10]:
from gensim import models
import gensim

In [11]:
sentences = models.word2vec.LineSentence("corpusSegDone.txt")
model = models.word2vec.Word2Vec(sentences, size=1, window=5, min_count=5, workers=1)
# model.wv.vocab

## pick user top 10 word ( frequency-based )

In [60]:
top10_vector = np.zeros(shape=(l,10))
from collections import defaultdict

# top10_vector

In [62]:
import operator
jieba.set_dictionary('jieba_txt/dict.txt.big.txt')
jieba.load_userdict("jieba_txt/userdict.txt")
for i in range(l):
#     print('\n user'+str(i+1))
    frequency = defaultdict(int)
    for j in range(len(grouped_data.iloc[i][1])):
        seglist = jieba.cut((grouped_data.iloc[i][1])[j])
        texts = [word for word in seglist if word not in stopwordset]
        for text in texts:
            frequency[text] += 1
    a = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True)
#     print(a)
    
    row = np.zeros(shape=(1,10))
    
    r = 10 if len(a) >= 10 else len(a)
    for x in range(r):
        if a[x][0] in model:
            show = float(model[a[x][0]])
#           print(show)
            row[0][x] = show
#     print(row)
    top10_vector[i] = row

Building prefix dict from /home/tp6han/gp_new/jieba_txt/dict.txt.big.txt ...
DEBUG:jieba:Building prefix dict from /home/tp6han/gp_new/jieba_txt/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.u0ddb98113e9461fac3a7658d9e24e45e.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.u0ddb98113e9461fac3a7658d9e24e45e.cache
Loading model cost 1.557 seconds.
DEBUG:jieba:Loading model cost 1.557 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [49]:
top10_vector.shape

(10, 10)

In [50]:
top10_vector

array([[ 2.71614385,  2.56558371,  2.54221654,  2.66731286,  2.65039468,
         2.44536662, -0.29142943,  2.67349625,  2.23602009,  2.65336442],
       [ 2.63390899,  2.40578985,  0.        ,  3.14465642,  2.39848042,
         2.67349625,  2.88766527,  2.692909  ,  2.59842253,  2.65061975],
       [ 2.63048625,  2.75842261,  2.64310718,  2.65039468,  2.66415906,
         1.67548323,  2.72610903,  2.54221654,  2.59842253,  2.63390899],
       [ 2.69077969,  1.39918911,  2.43555856,  2.62388778,  0.09946831,
         2.67135477,  2.11503196,  2.653687  ,  2.63390899,  0.        ],
       [ 2.65336442,  2.78188682,  2.40578985,  2.328444  ,  2.5715301 ,
         2.56276679,  2.15182495,  1.92687786,  2.50496817,  2.42080212],
       [ 2.44536662,  2.67133379,  2.56558371,  2.81371379,  2.41730785,
         2.75842261,  2.63390899,  0.        ,  2.71149707,  3.39983249],
       [ 3.14465642,  0.91635418,  2.67349625,  2.69656563,  2.75123358,
         2.67397594,  2.72200203,  2.70140648

## 分群

In [51]:
from sklearn import cluster, datasets

# KMeans 演算法
kmeans_fit = cluster.KMeans(n_clusters = 3,random_state=1170).fit(top10_vector)

# 印出分群結果
cluster_labels = kmeans_fit.labels_
print("分群結果 前10：")
print(cluster_labels[:10])
print("---")




分群結果 前10：
[0 0 0 2 0 0 0 2 0 1]
---


## 分群結果數量

In [52]:
unique, counts = np.unique(cluster_labels, return_counts=True)
dict(zip(unique, counts))


{0: 7, 1: 1, 2: 2}

In [18]:
grouped_data['category'] = cluster_labels

In [19]:
grouped_data.head(20)

,from,text,category
0,1000970273334199,找不到阿/安安/你有打LOL嗎/好吧，可以帶我嗎/安安/凸/Sor/嗯/約哪/有啊/你先約的...,1
1,1001070263323689,!?/唐寶寶/你那麼愛慕他ㄇ/嗨/先問問您期望本人的性別是?/了解/我是男的/抱歉沒有/請問...,1
2,1001967239901904,早/在床上發懶/爆/斷/魂/你在暗諷綠藻嘛/壞/話說我好餓ㄛ/窩不要這個qq/ㄋ早餐吃什麼/...,1
3,1003172849781848,甲/你要做我的0？/人生勝利組是不會來玩這個的/你一定不是c銘/也不是業王/葉你妹/被盜/其...,1
4,1003929993039131,吃噓/肥宅早/我們不是場外人/場外ㄈㄓ/我沒用了/安安/是喔/場外ㄈㄓ嗎/豪/斷/欸/在打手...,1
5,1006942526092443,安/是/靠北哈哈/是要說啥/呃/對呀/好/那我們來玩個遊戲/1188-1100/扣掉的要補零...,1
6,1007352782715651,嗨/......./看來是沒有/我是有差點想打/哈哈/真的離開喔/想說多打一句好了/哈哈/下...,1
7,1018989321532115,Hi/O.O/Ooo/黃色那隻/已知啊/Hi/還沒睡啊/不睡啊,1
8,1020089304775462,嗨/嗨/斷/嗨/哪裡人/挖/國外耶/天龍國人啊/感覺不同世界/嗨/安安 幾歲 住那 浪...,1
9,1020229148084642,安安,2


In [20]:
aaa = grouped_data.groupby('category')['text'].apply(lambda x: '\n\n'.join(x)).reset_index()

In [21]:
aaa.iloc[0]['text']

'闢咧?/小編?/囉/配對成功，您與陌生人連線了，可以開始聊天囉！！\n\nHi,/English?/My name is Wunan/XD/Fuck, Its U again XD/ennuyé/Who is it./Who is it./Who is it./u first plz/Connected./Japan No1/XDrz/English?/I need translater XD./こんばんは/XD/こんばんは/English?/(sad/TAIWAN No1/XD/Sure/English?/Oh Hi/English?/English?\n\n安/男孩女？/嗨/？/嗨/男還女？/嗨/請問你什麼星座/雙魚/你是女的還男的？/嗨/嗨/你好/太突然了吧/你是男的吧？/愛要大聲說/別悶在心裡/拜拜\n\n嗨/ㄩㄇ/你劍華/抓到了\n\n野/斷/到處都是場外/幹/安安/ㄩㄇ/ㄩㄇ/幹/笑死/ㄩㄇ/當然/安安/ㄩㄇ?/ㄩㄇ/野/幹怎麼都是場外的/http://m.imgur.com/jb4ya1c/ㄩㄇ/%%/閉嘴魯蛇/我卡卡/其實我是葉王/我勝利組/你好我叫郭峻佑/約嗎/我肌肉超大/閉嘴欠封鎖/安安/男?女?/我男/玩甲甲嗎/姿萱/安安/噁/原來你不是姿萱/您好麥當勞歡樂送/你好/我是男生喔/劍華警報/給ㄩㄇ/斷/幹又是劍華/壯/晚安劍華\n\n安安/約ㄇ/加我fb/Andy wang/讓你幸福的不要不要/你哪裡人/我中壢/晚上可以約浪漫/會排赤唐氏症嗎/有點甲/但是一樣ㄩ/讓你不要不要的/今晚約一波浪漫\n\n安安/蹲/?/這個好有趣喔/不知道怎麼做到的XD/對阿 可是在FB的視窗/搞到好像在跟機器人聊天喔/哈哈/不要\n\n真棒/嗨/場外？/==/==\n不早了/私cp?/約浪漫/鬧哦/雖然這是遊戲/私cp?/真棒/有蘿莉圖？/好/難道沒同好嗎/幼女最棒/最近fb在圖傳出去前就鎖了/好像類似圖搜的技術/害我沒辦法看蘿莉/太蚌了/甲/工尛/尛id/高一男/抱歉/騙/luke200092736/來\n\n你好/ㄅㄓ？/單戈哥/上車/不是甲/是哦/爆氣哦/釣魚哦/怕.jpg/我龍哥/不約/野/斷屁/噓\n\n嗨/有人ㄇ/我是彈彈堂堂主/你好/我騎山豬/你撞我？/？？？？？？/？？？？？？？？/ㄍㄋㄋ定溝ㄍㄧ喇/高雄我地盤/苓雅/來/好啊出來彎/你484旗農ㄉ

In [22]:
aaa.iloc[1]['text']

'找不到阿/安安/你有打LOL嗎/好吧，可以帶我嗎/安安/凸/Sor/嗯/約哪/有啊/你先約的啊/喔/輸入0088/我懶的打了/皮米屌/這你嗎？/魏癢癢/哪裡人?/住哪裡啦？/超癢/哪裡人?/好遠/其實還好/新竹/龍潭?/ㄏㄅㄨㄐㄅㄌㄗㄋ/三小/幹又ㄋ/掰掰/凹?/凸sor\n\n!?/唐寶寶/你那麼愛慕他ㄇ/嗨/先問問您期望本人的性別是?/了解/我是男的/抱歉沒有/請問您有玩龍族拼圖嗎？/我輸掉可以安慰我一下嗎/如果你安慰我我會很開心的/我好想尿尿/有點髒/而且要清很麻煩欸/乾你怎麼知道我是男的/What/Are you kidding me/真的嗎/下次試試看/我好想尿尿/這是舒服的事 不用怕/ㄋ會舒服啊ㄅ4我/不是啊誰跟你說我要用舔ㄉㄌ/用毛毛的棒子刷一刷R/好啦我要去洗澡ㄌ我女友在找我\n妹妹ㄅㄅ/我剛洗完澡 嗨/真的假的啊/有緣/啊 可惜 我還沒開始吹/好的/你真貼心/好那我先吹你等我一下XDD/回來ㄌ/紅豆餅是怎樣XDD/會嗎哈哈哈/哈哈哈哈別理他他怪人/(拍拍/超可愛的哈哈哈XDD/但看到桐人我必須噓!/好啦別難過/你難過我心痛/幹嘛這樣 我很真心欸/你心痛我心痛我更心痛/你也很讓我感到很窩心/ˊㄇˋ/你說ㄇ嗎/注音ㄇ啊/喔喔 那是注音的二聲跟四聲啊/我為什麼突然覺得有點可愛啊XDD/哈哈哈很可愛啊/不是會死很多腦細胞那還再打一次XDD/沒欸/喔喔對齁可以用複製的/我剛剛腦子當機哈哈哈/還在!!/別難過我會心痛QuQ/QAQ/話說你剛剛說重開機那句我還真的沒注意到欸/我的錯你懲罰我吧！！/沒有啦QAQ/可是我讓你難過了/罪該萬死QAQ/噢不懲罰我吧!/我會被罪惡感淹沒的/Oh/So sweet/你會早睡嗎/記得不要為了我睡不飽/我會難過/算晚吧/我習慣了/早睡不了/你習慣早睡嗎/沒辦法都玩手機玩到忘記時間哈哈/所以我都長不高QQ/可是我比差不多還矮很多QQ/你會笑我嗎/168QQ/對了我是男的 我們好像都不知道對方是男是女齁/哈哈哈哈/..你認真嗎/真的假的啊!?/靠腰嚇我一跳/哈哈哈我們剛剛還講了這麼多肉麻的話/對啊/我身高不像嗎QQ/偷你的圖哈哈哈/女生這樣也算超高吧/嗚嗚嗚嗚嗚嗚/我附近女生都160上下而已欸/矮矮的可愛/165快逼近我了/對我來說壓力大/那是你那麼高才能165剛剛好啊QQ/我快要沒機會了/嗚嗚嗚嗚嗚嗚/欸靠你怎麼猜這麼準/你這樣

In [23]:
aaa.iloc[2]['text']

'安安\n\nㄩㄇ\n\n8主/一堆場外\n\n覺得酷/?/好吃\n\n@@!/不是/XDDDDDD\n\n痾\n\n早安\n\n嗯？\n\n鉿/你/是/我網路有點慢喔\n\n男\n\n安\n\nLINE版爆炸啦😨😨😨/嗨/台北/what?/喔/嗨/安安 屁孩 天龍國 不浪漫/乖乖 別哭/應該的/幹 嚇死誰/哈哈哈哈 有夠笨/這張就夠嚇人了呀\n\n靠妖自己跟自己喔\n\n？\n\n180/75/男/沒/180/75/180/75\n\nHi/ㄩㄇ\n\n閉嘴魯蛇/我勝利組\n\n?/?\n\n嗨你好\n\n約浪漫\n\n哈哈/靠爸/掰\n\n安安\n\n哈嘍/？/哈哈好啊/嗨\n\n安安/??/我是阿銘\n\n有人嗎/哦哦哦哦哦！好神奇的東西\n\n嗨/怕/太樂觀了吧\n\n17男/安安/幹媽媽騙我\n\n🍔/嗨\n\n嗨/嗨/你好😂\n\n安安/安安\n\n.\n\n早安\n\n嗨..\n\n安安\n\n嗨/ㄩㄇ/豪\n\nㄩㄇ/嗨\n\nHi\n\n@@/嘿/尼哥\n\n嗨\n\nHi/Hi/Hi/Hi/哈哈哈\n\n安\n\nHi\n\n..\n\n嗨/講人話/我勝利組\n\n??/Hello/English please/></88\n\n嗨/男\n\nYee/阿羅哈\n\ni8主/ㄩㄇ？/ㄩ浪漫ㄇ？\n\n嗨/嗨/嗨嗨/嗨嗨嗨嗨嗨\n\n哈哈～科系呢？\n\n野\n\n87\n\n野/格\n\n.\n\n你好/野/?/沒事/沒事\n\n0800/0080\n\n野\n\nHI/妳好/男20/?\n\n安\n\n哈樓\U0010008d/哈囉\n\n野/嗨\n\n安/插上去\n\nㄤ/野/炸\n\nㄤ/4\n\n嗨\n\n？\n\nㄩㄇ\n\n幹 你霸主哦😧/等下 這是不是霸社專用app阿/怎麼遇都是/豪詭異哦/來/開唱/開唱/姿宣給你/好呀/嘿/嘿/其實./我也不知道他是誰 哈哈/我去另外ㄩ/88/ggwp?/幹 ㄩㄇ/想成功致富又快樂?/這她媽真的是霸社專用app耶/怎麼配都是/慘 先上車了/ㄩㄇ/不甲不甲/我讓你成功快樂又致富/ㄤㄤ/看實況:P\n\n你好\n\n安\n\n格/幹/幹對啦/幹我已經遇到五個場外了/最哭那種/8888888 找下一個目標/YEE/閉嘴 魯蛇 我勝利組/唐寶寶?/嗨/蛤/甚麼問題/閉嘴 魯蛇 我勝利組/幹/又場外/我以為/